ELM-FF

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# ------------------- Import Libraries -------------------
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load training and testing data
df_tr = pd.read_excel('/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs/PROJECT.xlsx', sheet_name="TR")
df_ts = pd.read_excel('/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs/PROJECT.xlsx', sheet_name="TS")

data_tr = df_tr.values
data_ts = df_ts.values

X_train = data_tr[:, :-1]
Y_train = data_tr[:, -1].reshape(-1, 1)
X_test = data_ts[:, :-1]
Y_test = data_ts[:, -1].reshape(-1, 1)

# ------------------- Normalize Data -------------------
x_scaler = StandardScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = StandardScaler()
Y_train = y_scaler.fit_transform(Y_train)
Y_test = y_scaler.transform(Y_test)

# Save scalers
save_dir = '/content/gdrive/MyDrive/Web and Data/Code/9. Hybrid_ELMs'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(x_scaler, os.path.join(save_dir, 'x_scaler.joblib'))
joblib.dump(y_scaler, os.path.join(save_dir, 'y_scaler.joblib'))

# ------------------- ELM Model -------------------
class ELM:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.W = np.random.randn(input_dim, hidden_dim)
        self.b = np.random.randn(hidden_dim)

    def hidden_output(self, X):
        H = np.dot(X, self.W) + self.b
        return np.tanh(H)

    def predict(self, X, beta):
        H = self.hidden_output(X)
        return np.dot(H, beta)

    def get_weights_dim(self):
        return self.hidden_dim * self.output_dim

# ------------------- Firefly Optimizer for ELM -------------------
class Firefly_ELM:
    def __init__(self, elm, X_train, y_train, X_test, y_test, y_scaler,
                 pop_size=30, max_iter=100, alpha=0.2, beta0=1, gamma=1):
        self.elm = elm
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.y_scaler = y_scaler

        self.pop_size = pop_size
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta0 = beta0
        self.gamma = gamma
        self.dim = elm.get_weights_dim()

        self.population = np.random.randn(pop_size, self.dim)
        self.fitness = np.array([self.evaluate(ind) for ind in self.population])
        self.best_idx = np.argmin(self.fitness)
        self.best_solution = self.population[self.best_idx].copy()
        self.best_score = self.fitness[self.best_idx]

    def evaluate(self, beta_flat):
        beta = beta_flat.reshape(self.elm.hidden_dim, self.elm.output_dim)
        y_pred = self.elm.predict(self.X_train, beta)
        return mean_squared_error(self.y_train, y_pred)

    def compute_metrics(self, beta_flat, X, y):
        beta = beta_flat.reshape(self.elm.hidden_dim, self.elm.output_dim)
        y_pred_scaled = self.elm.predict(X, beta)
        y_pred = self.y_scaler.inverse_transform(y_pred_scaled)
        y_true = self.y_scaler.inverse_transform(y)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        return rmse, r2

    def optimize(self):
        print(f"{'Iter':<6}{'Train RMSE':<12}{'Train R2':<10}{'Test RMSE':<12}{'Test R2':<10}")
        for t in range(self.max_iter):
            for i in range(self.pop_size):
                for j in range(self.pop_size):
                    if self.fitness[j] < self.fitness[i]:
                        r = np.linalg.norm(self.population[i] - self.population[j])
                        beta = self.beta0 * np.exp(-self.gamma * r ** 2)
                        step = beta * (self.population[j] - self.population[i]) + \
                               self.alpha * (np.random.rand(self.dim) - 0.5)
                        self.population[i] += step

            self.fitness = np.array([self.evaluate(ind) for ind in self.population])
            self.best_idx = np.argmin(self.fitness)
            self.best_solution = self.population[self.best_idx].copy()
            self.best_score = self.fitness[self.best_idx]

            train_rmse, train_r2 = self.compute_metrics(self.best_solution, self.X_train, self.y_train)
            test_rmse, test_r2 = self.compute_metrics(self.best_solution, self.X_test, self.y_test)
            print(f"{t+1:<6}{train_rmse:<12.4f}{train_r2:<10.4f}{test_rmse:<12.4f}{test_r2:<10.4f}")

        return self.best_solution

# ------------------- Run ELM + Firefly -------------------
input_dim = X_train.shape[1]
output_dim = 1
hidden_dim = 5
elm = ELM(input_dim, hidden_dim, output_dim)
fa = Firefly_ELM(elm, X_train, Y_train, X_test, Y_test, y_scaler,
                 pop_size=30, max_iter=100)
best_beta_flat = fa.optimize()
best_beta = best_beta_flat.reshape(hidden_dim, output_dim)

# ------------------- Final Evaluation -------------------
final_preds_scaled = elm.predict(X_test, best_beta)
final_preds = y_scaler.inverse_transform(final_preds_scaled)
Y_test_orig = y_scaler.inverse_transform(Y_test)

final_rmse = np.sqrt(mean_squared_error(Y_test_orig, final_preds))
final_r2 = r2_score(Y_test_orig, final_preds)

print(f"\n✅ Final Test RMSE: {final_rmse:.4f}")
print(f"✅ Final Test R²: {final_r2:.4f}")

# ------------------- Save Model -------------------
model_data = {
    'beta': best_beta,
    'W': elm.W,
    'b': elm.b,
    'input_dim': input_dim,
    'hidden_dim': hidden_dim,
    'output_dim': output_dim
}
model_path = os.path.join(save_dir, 'elm_fa_best_model.joblib')
joblib.dump(model_data, model_path)
print(f"✅ Model saved to: {model_path}")
